In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np


DEVICE = torch.device('cpu')
OUTPUT_SIZE = 2048

model = models.resnext50_32x4d(weights=models.ResNeXt50_32X4D_Weights.IMAGENET1K_V2)

extraction_layer = model._modules.get('avgpool')
model.to(DEVICE)
model.eval()

scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def get_vec(arg, model, extraction_layer):
    image = normalize(to_tensor(scaler(arg))).unsqueeze(0).to(DEVICE)
    result = torch.zeros(1, OUTPUT_SIZE, 1, 1)
    def copy_data(m, i, o):
        result.copy_(o.data)
    hooked = extraction_layer.register_forward_hook(copy_data)
    with torch.no_grad():
        model(image)
    hooked.remove()
    return result

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-1a0047aa.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-1a0047aa.pth
100%|██████████| 95.8M/95.8M [00:00<00:00, 156MB/s]


In [2]:
import arrow
import base64
import pandas as pd
from glob import iglob
from io import BytesIO
from os.path import basename
from PIL import Image

ROOT = '/kaggle/input/bee-detection-dataset'
TEST_IMAGES = ROOT + '/test/images'
TEST_LABELS = ROOT + '/test/labels'
THUMBNAIL_SIZE = (128, 128)
TRAIN_IMAGES = ROOT + '/train/images'
TRAIN_LABELS = ROOT + '/train/labels'
VALID_IMAGES = ROOT + '/valid/images'
VALID_LABELS = ROOT + '/valid/labels'


def embed(model, filename: str):
    with Image.open(fp=filename, mode='r') as image:
        return get_vec(arg=image.convert('RGB'), model=model, extraction_layer=extraction_layer).numpy().reshape(OUTPUT_SIZE,)

def label(filename: str, labels: str) -> int:
    label_file = labels + '/' + filename.replace('.jpg', '.txt')
    with open(file=label_file, mode='r') as input_fp:
        data = input_fp.read()
        if len(data.split()):
            return int(data.split()[0])
        else:
            return -1

def png(filename: str) -> str:
    with Image.open(fp=filename, mode='r') as image:
        buffer = BytesIO()
        # our images are pretty big; let's shrink the hover images to thumbnail size
        image.resize(size=THUMBNAIL_SIZE).convert('RGB').save(buffer, format='png')
        return 'data:image/png;base64,' + base64.b64encode(buffer.getvalue()).decode()

time_start = arrow.now()
valid_df = pd.DataFrame(data=[pd.Series(data=[basename(item), embed(model=model, filename=item), png(filename=item)],
                                        index=['item', 'value', 'png']) for item in iglob(VALID_IMAGES + '/*.jpg')])
valid_df['label'] = valid_df['item'].apply(func=label, args=(VALID_LABELS, ))
print('loaded valid: {} rows in {}'.format(len(valid_df), arrow.now() - time_start))
test_df = pd.DataFrame(data=[pd.Series(data=[basename(item), embed(model=model, filename=item), png(filename=item)],
                                        index=['item', 'value', 'png']) for item in iglob(TEST_IMAGES + '/*.jpg')])
test_df['label'] = test_df['item'].apply(func=label, args=(TEST_LABELS, ))
print('loaded test: {} rows in {}'.format(len(test_df), arrow.now() - time_start))
train_df = pd.DataFrame(data=[pd.Series(data=[basename(item), embed(model=model, filename=item), png(filename=item)],
                                        index=['item', 'value', 'png']) for item in iglob(TRAIN_IMAGES + '/*.jpg')])
train_df['label'] = train_df['item'].apply(func=label, args=(TRAIN_LABELS, ))
print('done in {}'.format(arrow.now() - time_start))

loaded valid: 1604 rows in 0:04:39.050900
loaded test: 836 rows in 0:07:03.159670
done in 0:23:21.454695


In [3]:
pd.concat(objs=[train_df, valid_df])['label'].value_counts().to_dict()

{0: 6115, -1: 1129}

In [4]:
import arrow
from umap import UMAP

time_start = arrow.now()
umap = UMAP(random_state=2024, verbose=False, n_jobs=1, low_memory=False, n_epochs=201)
train_df[['x', 'y']] = umap.fit_transform(X=train_df['value'].apply(func=pd.Series))
valid_df[['x', 'y']] = umap.transform(X=valid_df['value'].apply(func=pd.Series))
test_df[['x', 'y']] = umap.transform(X=test_df['value'].apply(func=pd.Series))
print('done with UMAP in {}'.format(arrow.now() - time_start))

done with UMAP in 0:00:56.419581
